**SpaceX Falcon 9 Birinci Aşama İniş Tahmini**

**Wikipedia’dan Falcon 9 ve Falcon Heavy Fırlatma Kayıtlarının Web Scraping Yöntemiyle Toplanması**

Bu laboratuvar çalışmasında, **Wikipedia** sayfasındaki *List of Falcon 9 and Falcon Heavy launches* başlıklı içerikten **Falcon 9’a ait tarihsel fırlatma kayıtlarını** toplamak için web scraping işlemi gerçekleştireceksiniz:

🔗 [https://en.wikipedia.org/wiki/List\_of\_Falcon\_9\_and\_Falcon\_Heavy\_launches](https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches)

---

### 🎯 Hedefler

**BeautifulSoup** kullanarak Falcon 9 fırlatma kayıtlarını web scrape edeceksiniz:

* Wikipedia’daki **Falcon 9 fırlatma kayıtları HTML tablosunu** çıkartmak
* Bu tabloyu ayrıştırmak (parse etmek) ve bir **Pandas DataFrame’e dönüştürmek**

---

### 🔧 İlk olarak, bu laboratuvar için gerekli kütüphaneleri içe aktaralım.


In [1]:
import sys  # Sistemle ilgili özelliklere erişmek için kullanılır (genellikle hata yönetimi veya yol ayarları için)

import requests  # Web sayfalarına HTTP istekleri göndermek için kullanılır (örneğin Wikipedia'dan veri almak için)
from bs4 import BeautifulSoup  # HTML ve XML dökümanlarını ayrıştırmak (parse etmek) için kullanılır
import re  # Düzenli ifadeler ile metinlerde arama ve eşleştirme işlemleri yapmak için kullanılır
import unicodedata  # Unicode karakterleri normalize etmek için kullanılır (özellikle özel karakterleri temizlemek için)
import pandas as pd  # Verileri tablo (DataFrame) yapısında işlemek ve analiz etmek için kullanılır


ve web kazıma (web scraping) ile elde edilen HTML tablosunu işlemeniz için size bazı yardımcı fonksiyonlar sağlayacağız.


In [2]:
def date_time(table_cells):
    """
    Bu fonksiyon, bir HTML tablo hücresinden tarih ve saat bilgilerini çıkarır.
    Girdi: table_cells — tablo hücresinin içeriği
    Çıktı: [tarih, saat] listesi
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    Bu fonksiyon, HTML tablo hücresinden booster (yükseltici) versiyonunu döndürür.
    Girdi: table_cells — tablo hücresinin içeriği
    Çıktı: booster versiyon adı (string)
    """
    out = ''.join([booster_version for i, booster_version in enumerate(table_cells.strings) if i % 2 == 0][0:-1])
    return out

def landing_status(table_cells):
    """
    Bu fonksiyon, HTML tablo hücresinden iniş durumunu döndürür.
    Girdi: table_cells — tablo hücresinin içeriği
    Çıktı: iniş durumu (örnek: 'Success', 'Failure' vb.)
    """
    out = [i for i in table_cells.strings][0]
    return out

def get_mass(table_cells):
    """
    Bu fonksiyon, yük kütlesini HTML hücresinden çıkarır.
    Eğer veri varsa, "kg" içeren kısmı alır.
    Girdi: table_cells — tablo hücresinin içeriği
    Çıktı: yük kütlesi (örnek: '5000 kg'), veri yoksa 0 döner
    """
    mass = unicodedata.normalize("NFKD", table_cells.text).strip()  # Unicode karakterleri temizle
    if mass:
        mass.find("kg")
        new_mass = mass[0:mass.find("kg")+2]
    else:
        new_mass = 0
    return new_mass

def extract_column_from_header(row):
    """
    Bu fonksiyon, tablo başlık satırından sütun adını çıkartır.
    HTML etiketlerini (br, a, sup) temizler ve kalan metni sütun adı olarak döndürür.
    Girdi: row — HTML tablo başlık hücresi
    Çıktı: sütun adı (string)
    """
    if (row.br):
        row.br.extract()  # <br> etiketini kaldır
    if row.a:
        row.a.extract()   # <a> bağlantısını kaldır
    if row.sup:
        row.sup.extract()  # üst simgeyi (superscript) kaldır

    colunm_name = ' '.join(row.contents)  # Kalan içerikleri birleştir

    # Sadece boş veya sayılardan oluşan başlıkları filtrele
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name


Laboratuvar görevlerinin tutarlılığını sağlamak için, sizden 9 Haziran 2021 tarihinde güncellenmiş olan **"List of Falcon 9 and Falcon Heavy launches"** Wikipedia sayfasının bir anlık görüntüsünden (snapshot) veri kazımanız istenecek.


In [3]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Sonraki adımda, yukarıdaki URL'den HTML sayfasını talep edin ve bir yanıt (response) nesnesi elde edin.

**GÖREV 1: Falcon9 fırlatma bilgilerini içeren Wiki sayfasını URL üzerinden isteyin**
    
İlk olarak, HTTP GET yöntemi kullanarak Falcon9 fırlatma sayfasının HTML içeriğini bir HTTP yanıtı olarak alacağız.


In [4]:
# verilen static_url ile requests.get() metodunu kullan
# yanıtı bir nesneye ata

# requests.get() ile HTTP isteği gönderiyoruz ve yanıtı 'response' değişkenine atıyoruz
response = requests.get(static_url)

# Yanıtın başarılı olup olmadığını kontrol etmek için isterseniz şunu yazabilirsiniz:
print(response.status_code)  # 200 ise başarılıdır

200


HTML yanıtından bir BeautifulSoup nesnesi oluştur.


In [5]:
# response.text ile HTTP isteğinden gelen HTML içeriğini alıyoruz
# BeautifulSoup ile bu HTML içeriğini ayrıştırıyoruz (parse ediyoruz)
soup = BeautifulSoup(response.text, 'html.parser')

Sayfa başlığını yazdırarak BeautifulSoup nesnesinin doğru oluşturulduğunu doğrula


In [6]:
# soup.title özniteliğini kullan
# Sayfa başlığını yazdırarak BeautifulSoup nesnesinin doğru oluşturulduğunu kontrol et
print(soup.title)

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>


**GÖREV 2: HTML tablo başlığından tüm sütun/değişken isimlerini çıkar**
    
Sonraki adımda, HTML tablosunun başlığından ilgili tüm sütun isimlerini toplamak istiyoruz.

İlk olarak, Wikipedia sayfasındaki tüm tabloları bulmaya çalışalım. Eğer BeautifulSoup kullanımı ile ilgili hafızanı tazelemek istersen, bu laboratuvarın sonunda verilen dış referans linkine bakabilirsin.


In [7]:
# BeautifulSoup nesnesinde `find_all` fonksiyonunu kullanarak, `table` öğelerini ara
# Sonucu `html_tables` adlı listeye ata

# soup nesnesinde tüm <table> elementlerini bul ve html_tables listesine ata
html_tables = soup.find_all('table')


Üçüncü tablodan itibaren hedef tablomuz başlıyor; burada gerçek fırlatma kayıtları bulunuyor.


In [10]:
# html_tables listesindeki üçüncü tabloyu seçiyoruz (indeks 2 çünkü indeksler 0'dan başlar)
first_launch_table = html_tables[2]

# Seçilen tabloyu ekrana yazdırarak içeriğini kontrol ediyoruz
print(first_launch_table)

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11"><span class="cite-bracket">[</span>b<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12"><span class="cite-bracket">[</span>c<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 

Sonraki adımda, sadece `<th>` öğeleri üzerinde döngü kurup sağlanan `extract_column_from_header()` fonksiyonunu kullanarak sütun isimlerini tek tek çıkarmamız gerekiyor.


In [11]:
column_names = []

# first_launch_table üzerinde find_all() fonksiyonunu kullanarak tüm <th> elemanlarını bul
th_elements = first_launch_table.find_all('th')

# Her bir <th> elemanı için extract_column_from_header() fonksiyonunu uygula ve sütun adını al
for th in th_elements:
    name = extract_column_from_header(th)
    # Sütun adı boş değilse ve uzunluğu sıfırdan büyükse column_names listesine ekle
    if name is not None and len(name) > 0:
        column_names.append(name)


Çıkarılan sütun isimlerini kontrol et


In [12]:
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


**GÖREV 3: Fırlatma HTML tablosunu çözümleyerek bir veri çerçevesi oluştur**
    
Önce önceki görevde çıkardığımız sütun isimlerini anahtar olarak kullanarak boş bir sözlük (dictionary) oluşturacağız. Daha sonra bu sözlük Pandas veri çerçevesine (dataframe) dönüştürülecek.


In [13]:
# Extracted column names kullanarak boş bir sözlük oluşturuluyor
launch_dict = dict.fromkeys(column_names)

# İlgisiz bir sütun olan 'Date and time ( )' kaldırılıyor
del launch_dict['Date and time ( )']

# Sözlükteki her anahtarın değeri boş liste olarak başlatılıyor
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []

# Yeni bazı sütunlar ekleniyor
launch_dict['Version Booster'] = []
launch_dict['Booster landing'] = []
launch_dict['Date'] = []
launch_dict['Time'] = []


Sonraki adımda, `launch_dict` sözlüğünü tablo satırlarından çıkarılan fırlatma kayıtları ile doldurmamız gerekiyor.

Genellikle, Wikipedia sayfalarındaki HTML tablolarında referans linkleri (örneğin B0004.1\[8]), eksik değerler (N/A \[e]) veya tutarsız biçimlendirme gibi beklenmedik ek açıklamalar ve çeşitli gürültüler olabilir.

Parçalama (parsing) sürecini kolaylaştırmak için aşağıda tamamlanmamış bir kod örneği verilmiştir. Bu kodu, TODO olarak işaretlenen yerleri tamamlayarak ya da kendi mantığınızı yazarak tüm fırlatma tablolarını ayrıştırabilirsiniz.


In [16]:
extracted_row = 0
# Her tabloyu sırayla incele
for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
    # Tablodaki tüm satırları al
    for rows in table.find_all("tr"):
        # İlk tablo başlığı, fırlatma numarası olup olmadığını kontrol et
        if rows.th:
            if rows.th.string:
                flight_number = rows.th.string.strip()
                flag = flight_number.isdigit()  # Sayı mı diye kontrol et
        else:
            flag = False
        # Satırdaki tüm hücreleri al
        row = rows.find_all('td')
        # Eğer satır bir fırlatma numarası içeriyorsa verileri kaydet
        if flag:
            extracted_row += 1
            
            # Flight Number (Fırlatma Numarası) değerini sözlüğe ekle
            launch_dict['Flight No.'].append(flight_number)
            
            datatimelist = date_time(row[0])
            
            # Date (Tarih) değerini sözlüğe ekle
            date = datatimelist[0].strip(',')
            launch_dict['Date'].append(date)
            
            # Time (Saat) değerini sözlüğe ekle
            time = datatimelist[1]
            launch_dict['Time'].append(time)
            
            # Booster version (Takım Versiyonu) değerini al ve sözlüğe ekle
            bv = booster_version(row[1])
            if not bv:
                bv = row[1].a.string
            launch_dict['Version Booster'].append(bv)
            
            # Launch Site (Fırlatma Alanı) değerini al ve sözlüğe ekle
            launch_site = row[2].a.string
            launch_dict['Launch site'].append(launch_site)
            
            # Payload (Yük) değerini al ve sözlüğe ekle
            payload = row[3].a.string
            launch_dict['Payload'].append(payload)
            
            # Payload Mass (Yük Kütlesi) değerini al ve sözlüğe ekle
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)
            
            # Orbit (Yörünge) değerini al ve sözlüğe ekle
            orbit = row[5].a.string
            launch_dict['Orbit'].append(orbit)
            
            # Customer (Müşteri) değerini al ve sözlüğe ekle
            customer = row[6].a.string if row[6].a else row[6].text.strip()  # Bazı hücrelerde link olmayabilir
            launch_dict['Customer'].append(customer)
            
            # Launch outcome (Fırlatma Sonucu) değerini al ve sözlüğe ekle
            launch_outcome = list(row[7].strings)[0]
            launch_dict['Launch outcome'].append(launch_outcome)
            
            # Booster landing (Takım İnişi) değerini al ve sözlüğe ekle
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)


Parçalanmış fırlatma kayıt değerlerini `launch_dict` içine doldurduktan sonra, bundan bir DataFrame oluşturabilirsiniz.


In [17]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

Şimdi bu veriyi bir sonraki bölüm için CSV dosyasına aktarabiliriz, ancak yanıtların tutarlı olması ve bu laboratuvarı tamamlamada zorluk yaşamanız durumunda, sonraki laboratuvarlarda bağımsız olmaları için sağlanan bir veri seti kullanılacaktır.


In [19]:
df.to_csv('spacex_web_scraped.csv', index=False)
